In [98]:
import numpy as np
import os
import sys
import datetime
import scipy as sp
import scipy.stats
from bisect import bisect
from rpy2.robjects.packages import importr
from rpy2.robjects.vectors import FloatVector
import math

def time_now():#return time
    curr_time = datetime.datetime.now()
    return curr_time.strftime("%c")

def Convert_wig_into_bp_coverage(extracted_coverage,extracted_3UTR_region,strand_info):
    bp_coverage = np.zeros(extracted_3UTR_region[-1] - extracted_3UTR_region[0])
    relative_start = extracted_3UTR_region[0]
    for i in range(len(extracted_coverage)):
        curr_region_start = extracted_3UTR_region[i] - relative_start
        curr_region_end = extracted_3UTR_region[i+1] - relative_start
        bp_coverage[curr_region_start:curr_region_end] = extracted_coverage[i]
    if strand_info == '-':
        bp_coverage = bp_coverage[::-1]
    
    return bp_coverage
        
def parse_cfgfile(cfg_file):
    '''Parse configure file
    '''
    Group1_Tophat_aligned_file=''
    Group2_Tophat_aligned_file=''
    output_directory=''
    Annotated_3UTR_file=''
    Output_result_file=''
    Num_least_in_group1_local=''
    Num_least_in_group2_local=''
    Coverage_cutoff_local = ''
    FDR_cutoff_local = ''
    Fold_change_cutoff_local = ''
    PDUI_cutoff_local = ''
    
    for line in open(cfg_file,'r'):
        if line[0] == '\n' or line[0] == '#':
            comments = line;
        else:
            line = line.rstrip();
            command = line.split('=');
            if command[0] == 'Group1_Tophat_aligned_Wig':
                Group1_Tophat_aligned_file = command[1].split(',');
            if command[0] == 'Group2_Tophat_aligned_Wig':
                Group2_Tophat_aligned_file = command[1].split(',');
            if command[0] == 'Output_directory':
                output_directory = command[1]
                if output_directory[-1] != '/':
                    output_directory += '/'
            if command[0] == 'Annotated_3UTR':
                Annotated_3UTR_file = command[1]
            if command[0] == 'Output_result_file':
                Output_result_file = command[1]
            
            ##Parameters
            if command[0] == 'Num_least_in_group1':
                Num_least_in_group1_local = command[1]
            if command[0] == 'Num_least_in_group2':
                Num_least_in_group2_local = command[1]
            if command[0] == 'Coverage_cutoff':
                Coverage_cutoff_local = command[1]
            if command[0] == 'FDR_cutoff':
                FDR_cutoff_local = command[1]
            if command[0] == 'Fold_change_cutoff':
                Fold_change_cutoff_local = command[1]
            if command[0] == 'PDUI_cutoff':
                PDUI_cutoff_local = command[1]
            
    
    if Group1_Tophat_aligned_file=='':
        print >> sys.stderr, "No Tophat aligned BAM file for group 1!"
        exit(1)
    if Group2_Tophat_aligned_file=='':
        print >> sys.stderr, "No Tophat aligned BAM file for group 2!"
        exit(1)
    if output_directory=='':
        print >> sys.stderr, "No output directory!"
        exit(1)
    if Annotated_3UTR_file=='':
        print >> sys.stderr, "No annotated 3' UTR file!"
        exit(1)
    if Output_result_file=='':
        print >> sys.stderr, "No result file name!"
        exit(1)
    return Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,output_directory,Annotated_3UTR_file,Output_result_file,Num_least_in_group1_local,Num_least_in_group2_local,Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,PDUI_cutoff_local

In [ ]:
def De_Novo_3UTR_Identification_Loading_Target_Wig_for_TCGA_Multiple_Samples_Main(argv=None):
    '''
    '''

In [104]:
if len(sys.argv) == 1:
    print("Please provide the configure file ...")
    exit(1)
cfg_file = "/storage2/huanglu/report/APA/test/adv_dapars/data_configureSRR4496792VSSRR4496781.txt"

In [105]:
cfg_file

'/storage2/huanglu/report/APA/test/adv_dapars/data_configureSRR4496792VSSRR4496781.txt'

In [106]:
print >> sys.stderr, "[%s] Start Analysis ..." % time_now()
Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,output_directory,Annotated_3UTR_file,Output_result_file,Num_least_in_group1_local,Num_least_in_group2_local,Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,PDUI_cutoff_local = parse_cfgfile(cfg_file)

num_group_1 = len(Group1_Tophat_aligned_file)
All_Sample_files = Group1_Tophat_aligned_file[:]
All_Sample_files.extend(Group2_Tophat_aligned_file)


global Num_least_in_group1
global Num_least_in_group2
global Coverage_cutoff
global FDR_cutoff
global Fold_change_cutoff
global PDUI_cutoff

if Num_least_in_group1_local != '':
    Num_least_in_group1 = float(Num_least_in_group1_local)
if Num_least_in_group2_local != '':
    Num_least_in_group2 = float(Num_least_in_group2_local)
if Coverage_cutoff_local != '':
    Coverage_cutoff = float(Coverage_cutoff_local)
if FDR_cutoff_local != '':
    FDR_cutoff = float(FDR_cutoff_local)
if Fold_change_cutoff_local != '':
    Fold_change_cutoff = float(Fold_change_cutoff_local)
if PDUI_cutoff_local != '':
    PDUI_cutoff = float(PDUI_cutoff_local)



[Sat 04 Aug 2018 12:40:53 PM ] Start Analysis ...


In [120]:
os.chdir("/storage2/huanglu/report/APA/test/adv_dapars")
Annotated_3UTR_file

'hg19_refseq_extracted_3UTR.bed'

In [121]:
##Prepare output directory
d = os.path.dirname(output_directory)
if not os.path.exists(d):
    os.makedirs(d)
temp_dir = d+'/tmp/'
if not os.path.exists(temp_dir):
    os.makedirs(temp_dir)
Output_all_prediction_file = output_directory+Output_result_file+'_result_temp.txt'
Output_result = open(Output_all_prediction_file, 'w')

num_samples = len(All_Sample_files)

##Debug
print >> sys.stderr, "[%s] Loading coverage ..." % time_now()
All_samples_Target_3UTR_coverages, All_samples_sequencing_depths, UTR_events_dict = Load_Target_Wig_files(All_Sample_files, Annotated_3UTR_file)
All_sample_coverage_weights = All_samples_sequencing_depths/np.mean(All_samples_sequencing_depths)
print >> sys.stderr, "[%s] Loading coverage finished ..." % time_now()
##Write the first line
first_line = ['Gene','fit_value','Predicted_Proximal_APA','Loci']
for i in range(num_group_1):
    curr_long_exp = 'A_%s_long_exp' % str(i+1)
    curr_short_exp = 'A_%s_short_exp' % str(i+1)
    curr_ratio ='A_%s_PDUI' % str(i+1)
    first_line.extend([curr_long_exp,curr_short_exp,curr_ratio])
for i in range(num_samples - num_group_1):
    curr_long_exp = 'B_%s_long_exp' % str(i+1)
    curr_short_exp = 'B_%s_short_exp' % str(i+1)
    curr_ratio ='B_%s_PDUI' % str(i+1)
    first_line.extend([curr_long_exp,curr_short_exp,curr_ratio])
first_line.append('PDUI_Group_diff')

[Sat 04 Aug 2018 12:57:05 PM ] Loading coverage ...
[Sat 04 Aug 2018 12:59:42 PM ] Loading coverage finished ...


In [124]:
All_samples_Target_3UTR_coverages[All_samples_Target_3UTR_coverages.keys()[1]]

[[[1, 2, 1, 2, 3, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 0],
  [45879584,
   45879632,
   45879640,
   45879704,
   45879705,
   45879708,
   45879780,
   45879781,
   45879782,
   45879852,
   45879952,
   45880018,
   45880027,
   45880094,
   45880220,
   45880294,
   45880296,
   45880323,
   45880370,
   45880399,
   45880475]],
 [[0,
   1,
   0,
   1,
   2,
   1,
   2,
   1,
   4,
   5,
   4,
   1,
   0,
   1,
   4,
   6,
   7,
   6,
   3,
   2,
   1,
   2,
   1,
   2,
   3,
   2,
   1,
   0,
   1,
   0,
   1,
   0],
  [45879584,
   45879608,
   45879684,
   45879706,
   45879766,
   45879782,
   45879841,
   45879842,
   45879859,
   45879893,
   45879917,
   45879933,
   45879969,
   45880023,
   45880035,
   45880053,
   45880090,
   45880099,
   45880111,
   45880129,
   45880166,
   45880182,
   45880204,
   45880213,
   45880214,
   45880258,
   45880289,
   45880290,
   45880317,
   45880336,
   45880337,
   45880394,
   45880475]]]

In [133]:
Output_result.writelines('\t'.join(first_line) + '\n')
totalresult=[]
for curr_3UTR_id in UTR_events_dict:
    curr_3UTR_structure = UTR_events_dict[curr_3UTR_id]
    region_start = curr_3UTR_structure[1]
    region_end   = curr_3UTR_structure[2]
    curr_strand  = curr_3UTR_structure[-2]
    UTR_pos = curr_3UTR_structure[-1]
    if curr_3UTR_id in All_samples_Target_3UTR_coverages:
        curr_3UTR_coverage_wig = All_samples_Target_3UTR_coverages[curr_3UTR_id]
        curr_3UTR_all_samples_bp_coverage = []
        for curr_sample_curr_3UTR_coverage_wig in curr_3UTR_coverage_wig: 
            curr_3UTR_curr_sample_bp_coverage = Convert_wig_into_bp_coverage(curr_sample_curr_3UTR_coverage_wig[0],curr_sample_curr_3UTR_coverage_wig[1],curr_strand)
            curr_3UTR_all_samples_bp_coverage.append(curr_3UTR_curr_sample_bp_coverage)

        select_mean_squared_error,selcted_break_point,UTR_abundances = De_Novo_3UTR_Coverage_estimation_Genome_for_TCGA_multiple_samples(curr_3UTR_all_samples_bp_coverage, region_start, region_end,curr_strand,All_sample_coverage_weights)


        if str(select_mean_squared_error) != "Na":
            Long_3UTR_exp_all = np.array(UTR_abundances[0])
            Short_3UTR_exp_all = np.array(UTR_abundances[1])
            num_non_zero = sum((Long_3UTR_exp_all + Short_3UTR_exp_all)>0)
            if num_non_zero == num_samples:
                All_Long_inclusion_ratios = []
                line_write = [curr_3UTR_id, "%.1f" % select_mean_squared_error, str(selcted_break_point), UTR_pos]
                for i in range(num_samples):
                    curr_sample_ratio = float(UTR_abundances[0][i])/(float(UTR_abundances[0][i]) + float(UTR_abundances[1][i]))##long 3'UTR percentage
                    All_Long_inclusion_ratios.append(curr_sample_ratio)
                    line_write.append("%.2f" % UTR_abundances[0][i])
                    line_write.append("%.2f" % UTR_abundances[1][i])
                    line_write.append("%.2f" % curr_sample_ratio)

                Group1_IR = All_Long_inclusion_ratios[:num_group_1]
                Group2_IR = All_Long_inclusion_ratios[num_group_1:]
                inclusion_ratio_Group_diff = np.mean(np.array(Group1_IR)) - np.mean(np.array(Group2_IR))

                line_write.append("%.2f" % inclusion_ratio_Group_diff)

                totalresult.append(line_write)

In [136]:
totalresult

[['NM_001127227|CD59|chr11|-',
  '21701.5',
  '33730397',
  'chr11:33724556-33731889',
  '39.04',
  '369.79',
  '0.10',
  '192.21',
  '1292.35',
  '0.13',
  '-0.03'],
 ['NM_017840|MRPL16|chr11|-',
  '381.1',
  '59573807',
  'chr11:59573608-59574305',
  '91.76',
  '0.00',
  '1.00',
  '74.00',
  '18.51',
  '0.80',
  '0.20'],
 ['NM_002437|MPV17|chr2|-',
  '262.6',
  '27532508',
  'chr2:27532360-27532849',
  '93.42',
  '6.54',
  '0.93',
  '161.15',
  '15.73',
  '0.91',
  '0.02'],
 ['NM_003099|SNX1|chr15|+',
  '125.1',
  '64430136',
  'chr15:64429767-64436433',
  '24.75',
  '43.28',
  '0.36',
  '16.47',
  '92.48',
  '0.15',
  '0.21'],
 ['NM_000491|C1QB|chr1|+',
  '387.3',
  '22987506',
  'chr1:22987305-22988029',
  '103.60',
  '0.00',
  '1.00',
  '46.34',
  '0.00',
  '1.00',
  '0.00'],
 ['NM_001326|CSTF3|chr11|-',
  '29.1',
  '33106389',
  'chr11:33106130-33106835',
  '18.99',
  '5.40',
  '0.78',
  '36.47',
  '1.17',
  '0.97',
  '-0.19'],
 ['NR_024108|PFKL|chr21|+',
  '1294.8',
  '45747074'

In [ ]:
print >> sys.stderr, "[%s] Filtering the result ..." % time_now()

Output_Motif_filtered_result_file = output_directory+Output_result_file+'_All_Prediction_Results.txt'
#UTR_APA_Result_filtering(Output_all_prediction_file,Genome_seq_fasta,Output_Motif_filtered_result_file)

DaPars_Filtering(Output_all_prediction_file, num_samples,num_group_1 ,Output_Motif_filtered_result_file)


try:
    os.remove(Output_all_prediction_file)
except OSError:
    pass

try:
    os.rmdir(temp_dir)
except OSError:
    pass



print >> sys.stderr, "[%s] Finished!" % time_now()

In [ ]:
#def DaPars_Filtering_debug():
def DaPars_Filtering(input_file, num_samples,num_group1 ,output_file):
    

In [ ]:
#cfg_file = 'CFIm25_Configure.txt'
#Group1_Tophat_aligned_file,Group2_Tophat_aligned_file,output_directory,Annotated_3UTR_file,Output_result_file,Num_least_in_group1_local,Num_least_in_group2_local,Coverage_cutoff_local,FDR_cutoff_local,Fold_change_cutoff_local,PDUI_cutoff_local = parse_cfgfile(cfg_file)

#input_file = 'CFIm25_KD_vs_Control_3UTR_All_prediction.txt'
#input_file = 'Wagner_3UTR_New_Nov_5_2012_All_prediction.txt'
#output_file = 'filtered.txt'
#num_samples = 2
#num_group1 = 1



#     if FDR_cutoff_local != '':
#         global FDR_cutoff
#         FDR_cutoff = FDR_cutoff_local
#         print FDR_cutoff
#     if PDUI_cutoff_local != '':
#         global PDUI_cutoff
#         PDUI_cutoff = PDUI_cutoff_local
#         print PDUI_cutoff





output_write = open(output_file,'w')

In [150]:
for line in totalresult:
    print(line)

['NM_000017|ACADS|chr12|+', '63.4', '121177501', 'chr12:121177099-121177811', '24.04', '23.10', '0.51', '36.52', '18.54', '0.66', '-0.15']
['NM_000018|ACADVL|chr17|+', '9241.2', '7128477', 'chr17:7128276-7128585', '340.96', '175.09', '0.66', '400.39', '245.05', '0.62', '0.04']
['NM_000019|ACAT1|chr11|+', '451.4', '108018256', 'chr11:108017997-108018891', '6.29', '71.22', '0.08', '6.98', '110.99', '0.06', '0.02']
['NM_000021|PSEN1|chr14|+', '118.7', '73687075', 'chr14:73685842-73690399', '7.63', '33.02', '0.19', '11.00', '62.72', '0.15', '0.04']
['NM_000022|ADA|chr20|-', '39.0', '43248287', 'chr20:43248163-43248488', 'NA', 'NA', 'NA', '25.08', '19.83', '0.56', 'NA']
['NM_000029|AGT|chr1|-', '12089.4', '230838542', 'chr1:230838272-230839075', '418.96', '274.34', '0.60', '600.19', '370.81', '0.62', '-0.02']
['NM_000031|ALAD|chr9|-', '1284.8', '116150440', 'chr9:116148592-116150641', '105.68', '0.00', '1.00', '145.07', '0.00', '1.00', '0.0']
['NM_000034|ALDOA|chr16|+', '36485.5', '30081639

['NM_004516|ILF3|chr19|+', '173.3', '10795293', 'chr19:10795092-10796443', '55.35', '0.00', '1.00', '83.28', '0.00', '1.00', '0.0']
['NM_004517|ILK|chr11|+', '586.9', '6631894', 'chr11:6631693-6632048', '124.60', '47.96', '0.72', '186.86', '77.92', '0.71', '0.01']
['NM_004520|KIF2A|chr5|+', '60.6', '61682163', 'chr5:61681311-61683011', '28.21', '3.78', '0.88', '27.33', '27.08', '0.50', '0.38']
['NM_004521|KIF5B|chr10|-', '6247.6', '32300243', 'chr10:32297938-32300444', '158.81', '0.00', '1.00', '234.59', '0.00', '1.00', '0.0']
['NM_004522|KIF5C|chr2|+', '15702.0', '149879793', 'chr2:149879592-149883273', '328.98', '0.00', '1.00', '488.03', '0.00', '1.00', '0.0']
['NM_004528|MGST3|chr1|+', '5054.6', '165624806', 'chr1:165624605-165625372', '19.67', '210.74', '0.09', '29.32', '452.74', '0.06', '0.03']
['NM_004531|MOCS2|chr5|-', '406.9', '52394190', 'chr5:52393895-52394497', '112.82', '13.71', '0.89', '191.89', '77.83', '0.71', '0.18']
['NM_004537|NAP1L1|chr12|-', '1202.2', '76440022', 'c

['NM_025132|WDR19|chr4|+', '24.7', '39287331', 'chr4:39287107-39287430', 'NA', 'NA', 'NA', '17.93', '23.84', '0.43', 'NA']
['NM_025135|FHOD3|chr18|+', '55.5', '34359612', 'chr18:34359411-34360018', 'NA', 'NA', 'NA', '44.59', '0.00', '1.00', 'NA']
['NM_025137|SPG11|chr15|-', '155.5', '44855158', 'chr15:44855001-44855499', '49.57', '2.12', '0.96', '83.36', '7.56', '0.92', '0.04']
['NM_025140|CCDC92|chr12|-', '3194.7', '124422176', 'chr12:124420955-124422377', '120.35', '0.00', '1.00', '140.25', '0.00', '1.00', '0.0']
['NM_025141|TM2D3|chr15|-', '977.5', '102182646', 'chr15:102182049-102182847', '60.25', '0.00', '1.00', '127.98', '0.00', '1.00', '0.0']
['NM_025146|NAA50|chr3|-', '513.3', '113437870', 'chr3:113435307-113440784', '2.54', '30.00', '0.08', '6.97', '79.69', '0.08', '0.0']
['NM_025147|COQ10B|chr2|+', '44.1', '198338789', 'chr2:198338481-198339851', 'NA', 'NA', 'NA', '40.03', '5.36', '0.88', 'NA']
['NM_025149|ACSF2|chr17|+', '187.1', '48552064', 'chr17:48551863-48552200', '27.12

In [144]:
len(totalresult)

6639

In [157]:
num_line = 0
num_group1=1
result_dict = {}
All_P_values = []
Selected_events_id = []
All_mean_abundance = []

for line in totalresult:
    if num_line > 0:
        fields = line
        group1_coverages = np.zeros(2)
        group2_coverages = np.zeros(2)
        num_group1_pass = 0
        group1_PDUIs = 0
        for i in range(num_group1):
            curr_long = fields[4+i*3]
            curr_short = fields[5+i*3]
            if curr_long != 'NA':
                curr_long  = float(curr_long)
                curr_short = float(curr_short)
                if curr_long + curr_short >= Coverage_cutoff:
                    group1_PDUIs = group1_PDUIs + float(fields[6+i*3])
                    num_group1_pass += 1
                    group1_coverages[0] = group1_coverages[0] + curr_long  
                    group1_coverages[1] = group1_coverages[1] + curr_short
                else:
                    fields[4+i*3] = 'NA'
                    fields[5+i*3] = 'NA'
                    fields[6+i*3] = 'NA'


        num_group2_pass = 0
        group2_PDUIs = 0
        for i in range(num_samples - num_group1):
            curr_long = fields[4+(i+num_group1)*3]
            curr_short = fields[5+(i+num_group1)*3]
            if curr_long != 'NA':
                curr_long  = float(curr_long)
                curr_short = float(curr_short)
                if curr_long + curr_short >= Coverage_cutoff:
                    group2_PDUIs = group2_PDUIs + float(fields[6+(i+num_group1)*3])
                    num_group2_pass += 1
                    group2_coverages[0] = group2_coverages[0] + curr_long  
                    group2_coverages[1] = group2_coverages[1] + curr_short
                else:
                    fields[4+(i+num_group1)*3] = 'NA'
                    fields[5+(i+num_group1)*3] = 'NA'
                    fields[6+(i+num_group1)*3] = 'NA'



        if num_group1_pass >= Num_least_in_group1 and num_group2_pass >= Num_least_in_group2:
            Final_group_diff = str(group1_PDUIs/num_group1_pass - group2_PDUIs/num_group2_pass)

            All_mean_abundance.append([group1_PDUIs/num_group1_pass, group2_PDUIs/num_group2_pass])

            fields[-1] = str(Final_group_diff)
            ratio_val,P_val = sp.stats.fisher_exact([group1_coverages/num_group1_pass,group2_coverages/num_group2_pass])

            All_P_values.append(P_val)
            Selected_events_id.append(fields[0])
            #print P_val
            #print ratio_val
            result_dict[fields[0]] = fields
        else:
            fields[-1] = 'NA'




    else:
        first_line = line     

    num_line += 1

In [158]:
for i in result_dict.keys():
    print(result_dict[i])

['NM_001316|CSE1L|chr20|+', '96.9', '47713087', 'chr20:47712886-47713497', '24.93', '20.72', '0.55', '56.75', '44.19', '0.56', '-0.01']
['NM_017411|SMN2|chr5|+', '485.2', '69373047', 'chr5:69372846-69373422', '55.13', '14.93', '0.79', '197.87', '30.61', '0.87', '-0.08']
['NM_001256732|SSBP2|chr5|-', '21.9', '80715791', 'chr5:80713179-80716352', '3.50', '26.66', '0.12', '3.46', '45.59', '0.07', '0.05']
['NM_021626|SCPEP1|chr17|+', '46.0', '55083935', 'chr17:55083534-55084129', '22.92', '12.18', '0.65', '36.55', '17.96', '0.67', '-0.02']
['NM_001655|ARCN1|chr11|+', '2526.1', '118471541', 'chr11:118471340-118473747', '43.53', '0.00', '1.00', '110.81', '0.00', '1.00', '0.0']
['NM_015972|POLR1D|chr13|+', '347.7', '28197365', 'chr13:28197012-28197590', '44.06', '11.38', '0.79', '105.76', '20.83', '0.84', '-0.05']
['NM_001136052|CPT1C|chr19|+', '183.5', '50216901', 'chr19:50216677-50216988', '87.94', '0.00', '1.00', '84.54', '0.00', '1.00', '0.0']
['NM_006224|PITPNA|chr17|-', '1229.6', '14236

['NM_002136|HNRNPA1|chr12|+', '23970.0', '54678771', 'chr12:54678333-54679030', '305.11', '0.00', '1.00', '480.66', '48.64', '0.91', '0.09']
['NM_032090|PCDHGA10|chr5|+', '7.6', '140792944', 'chr5:140792743-140795295', '184.61', '0.00', '1.00', '130.92', '0.00', '1.00', '0.0']
['NM_173854|SLC41A1|chr1|-', '1295.2', '205760645', 'chr1:205758221-205760846', '67.92', '0.00', '1.00', '105.22', '0.00', '1.00', '0.0']
['NM_032272|MAF1|chr8|+', '320.9', '145162329', 'chr8:145161991-145162515', '120.85', '8.45', '0.93', '155.69', '37.71', '0.81', '0.12']
['NM_001376|DYNC1H1|chr14|+', '2545.8', '102517033', 'chr14:102516772-102517135', '269.01', '0.00', '1.00', '476.62', '41.83', '0.92', '0.08']
['NM_019065|NECAB2|chr16|+', '261.1', '84036124', 'chr16:84035923-84036379', '168.08', '3.44', '0.98', '129.39', '21.11', '0.86', '0.12']
['NM_170721|MSI2|chr17|+', '284.8', '55709353', 'chr17:55709152-55710544', '68.46', '0.00', '1.00', '114.57', '0.00', '1.00', '0.0']
['NM_021129|PPA1|chr10|-', '1464.

['NM_006527|SLBP|chr4|-', '1040.4', '1695239', 'chr4:1694527-1695440', '40.90', '0.00', '1.00', '167.51', '0.00', '1.00', '0.0']
['NR_021490|FLJ42709|chr5|-', '5.3', '92879284', 'chr5:92877578-92879730', '38.80', '4.08', '0.90', '49.19', '0.00', '1.00', '-0.1']
['NM_006216|SERPINE2|chr2|-', '11953.8', '224840049', 'chr2:224839765-224840621', '386.24', '133.98', '0.74', '937.80', '393.48', '0.70', '0.04']
['NM_004286|GTPBP1|chr22|+', '91.7', '39128212', 'chr22:39126617-39129592', '18.98', '17.94', '0.51', '38.20', '11.18', '0.77', '-0.26']
['NM_145687|MAP4K4|chr2|+', '7366.7', '102509194', 'chr2:102507628-102511152', '112.64', '9.25', '0.92', '209.92', '124.10', '0.63', '0.29']
['NM_004711|SYNGR1|chr22|+', '693.1', '39777902', 'chr22:39777701-39781593', '78.54', '0.00', '1.00', '60.36', '0.00', '1.00', '0.0']
['NM_003367|USF2|chr19|+', '5575.8', '35770459', 'chr19:35770070-35770718', '348.33', '131.73', '0.73', '440.82', '159.44', '0.73', '0.0']
['NM_006353|HMGN4|chr6|+', '254.4', '2654

In [154]:
Selected_events_id

['NM_017840|MRPL16|chr11|-',
 'NM_002437|MPV17|chr2|-',
 'NM_003099|SNX1|chr15|+',
 'NM_000491|C1QB|chr1|+',
 'NR_024108|PFKL|chr21|+',
 'NM_004740|TIAF1|chr17|-',
 'NM_005125|CCS|chr11|+',
 'NM_001114748|TMEM240|chr1|-',
 'NM_004417|DUSP1|chr5|-',
 'NM_001190274|FBXO11|chr2|-',
 'NM_014866|SEC16A|chr9|-',
 'NM_000543|SMPD1|chr11|+',
 'NM_015260|SIN3B|chr19|+',
 'NM_032111|MRPL14|chr6|-',
 'NM_024047|NUDT9|chr4|+',
 'NR_037619|CMTM3|chr16|+',
 'NM_032552|DAB2IP|chr9|+',
 'NM_004309|ARHGDIA|chr17|-',
 'NM_001025|RPS23|chr5|-',
 'NM_006541|GLRX3|chr10|+',
 'NM_012423|RPL13A|chr19|+',
 'NM_031481|SLC25A18|chr22|+',
 'NM_015137|EFR3A|chr8|+',
 'NM_001164712|AMT|chr3|-',
 'NM_014463|LSM3|chr3|+',
 'NM_001946|DUSP6|chr12|-',
 'NM_001136015|ANXA2|chr15|-',
 'NM_182931|MLL5|chr7|+',
 'NM_002753|MAPK10|chr4|-',
 'NM_020248|CTNNBIP1|chr1|-',
 'NM_003179|SYP|chrX|-',
 'NM_014949|KIAA0907|chr1|-',
 'NM_012385|NUPR1|chr16|-',
 'NM_005717|ARPC5|chr1|-',
 'NM_024722|ACBD4|chr17|+',
 'NM_005131|THOC1|

In [155]:
##Filtering
stats = importr('stats')
All_p_adjust = stats.p_adjust(FloatVector(All_P_values), method = 'BH')
first_line.insert(-1,'Group_A_Mean_PDUI')
first_line.insert(-1,'Group_B_Mean_PDUI')
first_line.extend(['P_val','adjusted.P_val','Pass_Filter'])

In [156]:
All_p_adjust

R object with classes: ('numeric',) mapped to:
<FloatVector - Python:0x7fabad9e5488 / R:0x55a66dfada80>
[0.000079, 1.000000, 0.016852, ..., 1.000000, 0.000467, 1.000000]

In [ ]:
output_write.writelines('\t'.join(first_line)+'\n')

In [ ]:
for curr_event_id in result_dict:
    mean_PDUI_group1 = 'NA'
    mean_PDUI_group2 = 'NA'
    curr_P_val = 'NA'
    curr_FDR_val = 'NA'
    Pass_filter = 'N'
    curr_fields = result_dict[curr_event_id]
    if curr_event_id in Selected_events_id:
        sel_ind = Selected_events_id.index(curr_event_id)
        curr_P_val = str(All_P_values[sel_ind])
        curr_FDR_val = str(All_p_adjust[sel_ind])

        mean_PDUI_group1 = All_mean_abundance[sel_ind][0]
        mean_PDUI_group2 = All_mean_abundance[sel_ind][1]


        if float(curr_FDR_val) <= FDR_cutoff and abs(float(curr_fields[-1]))>=PDUI_cutoff and abs(math.log((mean_PDUI_group1+1e-5)/(mean_PDUI_group2+1e-5),2))>=Fold_change_cutoff:
            Pass_filter = 'Y'

    curr_fields.insert(-1,str(mean_PDUI_group1))
    curr_fields.insert(-1,str(mean_PDUI_group2))
    curr_fields.append(curr_P_val)
    curr_fields.append(curr_FDR_val)
    curr_fields.append(Pass_filter)

    output_write.writelines('\t'.join(curr_fields) +'\n')


output_write.close()

In [ ]:
  
def get_version():
    return "0.9.1"

In [111]:
def De_Novo_3UTR_Coverage_estimation_Genome_for_TCGA_multiple_samples(All_Samples_curr_3UTR_coverages, UTR_start, UTR_end,curr_strand,weight_for_second_coverage):
    '''For UTR-APA new
       Load one chromosome by chromosome
       Just for TCGA data analysis. So no peak evenness checking
       Jan-17-2013
       2-28-2013
    '''
    coverage_threshold = 20
    search_point_start     = 200
    search_point_end       = int(abs((UTR_end - UTR_start))*0.1)
    num_samples = len(All_Samples_curr_3UTR_coverages)
    ##read coverage
    Region_Coverages = []
    Region_mean_Coverages = []
    Region_first_100_coverage_all_samples = []
    for i in range(num_samples):
        curr_Region_Coverage_raw = All_Samples_curr_3UTR_coverages[i]##strand is reversed in load
        curr_Region_Coverage = curr_Region_Coverage_raw/weight_for_second_coverage[i] ##加权校正后的coverage
        Region_mean_Coverages.append(np.mean(curr_Region_Coverage_raw))
        Region_Coverages.append(curr_Region_Coverage) 
        curr_first_100_coverage = np.mean(curr_Region_Coverage_raw[0:99])
        Region_first_100_coverage_all_samples.append(curr_first_100_coverage)
    if sum(np.array(Region_first_100_coverage_all_samples) >= coverage_threshold) >= num_samples and UTR_end - UTR_start >= 150:
        if curr_strand == "+":
            search_region = range(UTR_start+search_point_start, UTR_end-search_point_end+1)
        else:
            search_region = range(UTR_end - search_point_start, UTR_start+search_point_end-1, -1)
        
        search_region_start = search_point_start
        search_region_end   = UTR_end - UTR_start - search_point_end
        Mean_squared_error_list  = []
        Estimated_3UTR_abundance_list = []
        for curr_point in range(search_region_start, search_region_end+1):
            curr_search_point = curr_point
            All_samples_result = [[],[],[]]
            for curr_sample_region_coverage in Region_Coverages: #每个个体在该区域的各个点的coverage的list
                Mean_Squared_error,Long_UTR_abun,Short_UTR_abun = Estimation_abundance(curr_sample_region_coverage, curr_search_point) ##看整个UTR的覆盖度和单个点之间的覆盖度关系
                All_samples_result[0].append(Mean_Squared_error)
                All_samples_result[1].append(Long_UTR_abun)
                All_samples_result[2].append(Short_UTR_abun)
            
            Mean_Squared_error = np.mean(np.array(All_samples_result[0]))
            Mean_squared_error_list.append(Mean_Squared_error)
            Estimated_3UTR_abundance_list.append([All_samples_result[1],All_samples_result[2]])

			#[[tissue1_long_utr,tissue2_long_utr,tissue3_long_utr][tissue1_short_utr,tissue2_short_utr,tissue3_short_utr]]

        if len(Mean_squared_error_list) > 0: ##如果点存在则一般长度大于零
            min_ele_index = Mean_squared_error_list.index(min(Mean_squared_error_list))
            
            select_mean_squared_error = Mean_squared_error_list[min_ele_index]
            UTR_abundances = Estimated_3UTR_abundance_list[min_ele_index]
            selcted_break_point = search_region[min_ele_index]
            
        else:
            select_mean_squared_error = 'Na'
            UTR_abundances = 'Na'
            selcted_break_point = 'Na'
        
    else:
        select_mean_squared_error = 'Na' ##未通过coverage cutoff且长度小于150bp
        UTR_abundances = 'Na'
        selcted_break_point = 'Na'
    
    return select_mean_squared_error,selcted_break_point,UTR_abundances

In [112]:
def Estimation_abundance(Region_Coverage, break_point):
    Long_UTR_abun  = np.mean(Region_Coverage[break_point:])
    Short_UTR_abun = np.mean(Region_Coverage[0:break_point] - Long_UTR_abun)
    if Short_UTR_abun < 0:
        Short_UTR_abun = 0
    Coverage_diff  = Region_Coverage[0:break_point] - Long_UTR_abun - Short_UTR_abun
    Coverage_diff= np.append(Coverage_diff, Region_Coverage[break_point:] - Long_UTR_abun)#断点前丰度list减（长+短）和断点后丰度list减（长）两个list之间的SE
    Mean_Squared_error = np.mean(Coverage_diff**2)
    
    return Mean_Squared_error,Long_UTR_abun,Short_UTR_abun
 

In [ ]:
   
def Load_Target_Wig_files(All_Wig_files, UTR_Annotation_file):

In [126]:
All_Wig_files=All_Sample_files
UTR_Annotation_file=Annotated_3UTR_file
UTR_events_dict = {}
All_Samples_Total_depth = []
for line in open(UTR_Annotation_file,'r'):
    fields = line.strip('\n').split('\t')
    curr_chr = fields[0]
    region_start = fields[1]
    region_end   = fields[2]
    curr_strand  = fields[-1]
    UTR_pos = "%s:%s-%s" % (curr_chr, region_start, region_end)
    end_shift = int(round(abs(int(region_start) - int(region_end)) * 0.2))
    if curr_strand == '+':
        region_end = str(int(region_end) - end_shift)
    else:
        region_start = str(int(region_start) + end_shift)
    region_start = int(region_start) + 1
    region_end   = int(region_end) - 1
    if region_start + 50 < region_end:
        UTR_events_dict[fields[3]] = [fields[0],region_start,region_end,fields[-1],UTR_pos]

In [127]:
UTR_events_dict

{'NM_001127227|CD59|chr11|-': ['chr11',
  33726024,
  33731888,
  '-',
  'chr11:33724556-33731889'],
 'NR_045674|CLIC5|chr6|-': ['chr6',
  45879584,
  45880475,
  '-',
  'chr6:45879360-45880476'],
 'NM_024913|C7orf58|chr7|+': ['chr7',
  120935495,
  120937096,
  '+',
  'chr7:120935494-120937498'],
 'NR_045406|HIF1A-AS2|chr14|-': ['chr14',
  62215144,
  62215806,
  '-',
  'chr14:62214977-62215807'],
 'NM_017840|MRPL16|chr11|-': ['chr11',
  59573748,
  59574304,
  '-',
  'chr11:59573608-59574305'],
 'NM_001005201|OR8H3|chr11|+': ['chr11',
  55889850,
  55890598,
  '+',
  'chr11:55889849-55890787'],
 'NM_002437|MPV17|chr2|-': ['chr2',
  27532459,
  27532848,
  '-',
  'chr2:27532360-27532849'],
 'NR_031570|MIR1301|chr2|-': ['chr2',
  25551526,
  25551589,
  '-',
  'chr2:25551509-25551590'],
 'NM_003099|SNX1|chr15|+': ['chr15',
  64429768,
  64435099,
  '+',
  'chr15:64429767-64436433'],
 'NM_014687|KIAA0226|chr3|-': ['chr3',
  197399022,
  197402066,
  '-',
  'chr3:197398259-197402067'],
 

In [128]:
##Load coverage for all samples
All_samples_extracted_3UTR_coverage_dict = {}
for curr_wig_file in All_Wig_files:
    curr_sample_All_chroms_coverage_dict = {}
    num_line = 0
    cur_sample_total_depth = 0
    for line in open(curr_wig_file,'r'):
        if '#' not in line and line[0:3] == 'chr':
            fields = line.strip('\n').split('\t')
            chrom_name = fields[0]
            region_start = int(fields[1])
            region_end = int(fields[2])
            cur_sample_total_depth += int(float(fields[-1])) * (region_end - region_start)
            if chrom_name not in curr_sample_All_chroms_coverage_dict:
                curr_sample_All_chroms_coverage_dict[chrom_name] = [[0],[0]]
            if region_start > curr_sample_All_chroms_coverage_dict[chrom_name][0][-1]:
                curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_start)
                curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
            curr_sample_All_chroms_coverage_dict[chrom_name][0].append(region_end)
            curr_sample_All_chroms_coverage_dict[chrom_name][1].append(int(float(fields[-1])))
        num_line += 1
    curr_sample_All_chroms_coverage_dict[chrom_name][1].append(0)
    All_Samples_Total_depth.append(cur_sample_total_depth)
    for curr_3UTR_event_id in UTR_events_dict:
        curr_3UTR_structure = UTR_events_dict[curr_3UTR_event_id]
        curr_chr = curr_3UTR_structure[0]
        if curr_chr in curr_sample_All_chroms_coverage_dict:
            curr_chr_coverage = curr_sample_All_chroms_coverage_dict[curr_chr]
            region_start = curr_3UTR_structure[1]
            region_end = curr_3UTR_structure[2]
            left_region_index = bisect(curr_chr_coverage[0],region_start)
            right_region_index = bisect(curr_chr_coverage[0],region_end)

            extracted_coverage = curr_chr_coverage[1][left_region_index:right_region_index+1]
            extracted_3UTR_region = curr_chr_coverage[0][left_region_index:right_region_index]
            extracted_3UTR_region.insert(0,region_start)
            extracted_3UTR_region.append(region_end)
            if curr_3UTR_event_id not in All_samples_extracted_3UTR_coverage_dict:
                All_samples_extracted_3UTR_coverage_dict[curr_3UTR_event_id] = []
            All_samples_extracted_3UTR_coverage_dict[curr_3UTR_event_id].append([extracted_coverage,extracted_3UTR_region])

In [131]:
All_samples_extracted_3UTR_coverage_dict[All_samples_extracted_3UTR_coverage_dict.keys()[1]]

[[[1, 2, 1, 2, 3, 2, 1, 0, 1, 0, 1, 2, 1, 0, 1, 2, 1, 2, 1, 0],
  [45879584,
   45879632,
   45879640,
   45879704,
   45879705,
   45879708,
   45879780,
   45879781,
   45879782,
   45879852,
   45879952,
   45880018,
   45880027,
   45880094,
   45880220,
   45880294,
   45880296,
   45880323,
   45880370,
   45880399,
   45880475]],
 [[0,
   1,
   0,
   1,
   2,
   1,
   2,
   1,
   4,
   5,
   4,
   1,
   0,
   1,
   4,
   6,
   7,
   6,
   3,
   2,
   1,
   2,
   1,
   2,
   3,
   2,
   1,
   0,
   1,
   0,
   1,
   0],
  [45879584,
   45879608,
   45879684,
   45879706,
   45879766,
   45879782,
   45879841,
   45879842,
   45879859,
   45879893,
   45879917,
   45879933,
   45879969,
   45880023,
   45880035,
   45880053,
   45880090,
   45880099,
   45880111,
   45880129,
   45880166,
   45880182,
   45880204,
   45880213,
   45880214,
   45880258,
   45880289,
   45880290,
   45880317,
   45880336,
   45880337,
   45880394,
   45880475]]]

In [ ]:
return All_samples_extracted_3UTR_coverage_dict,np.array(All_Samples_Total_depth),UTR_events_dict

In [ ]:
##Default Parameters
Num_least_in_group1 = 1
Num_least_in_group2 = 1
Coverage_cutoff = 30

FDR_cutoff = 0.05
PDUI_cutoff = 0.2
Fold_change_cutoff = 0.59 #1.5 fold change

if __name__ == '__main__':
    '''Non-paralle version for matched tumor-normal 3'UTR Identification.
       5-26-2013
    '''
    De_Novo_3UTR_Identification_Loading_Target_Wig_for_TCGA_Multiple_Samples_Main(sys.argv)
    
    ##debug
    #DaPars_Filtering_debug()
    